In [15]:
from collections import Counter, defaultdict
from modules.machine_learning import split_data, precision, recall, f1_score
import math, random, re, glob

# Naive Bayes

Uma rede social não é muito boa se as pessoas não podem interagir. Assim, a *DataSciencester* tem um recurso popular que permite que os membros enviem mensagens para outros membros. E enquanto a maioria de seus membros são cidadãos responsáveis que enviam apenas mensagens educadas do tipo "como vai?", alguns canalhas insistentemente enviam spam a outros membros sobre esquemas de enriquecimento, produtos farmacêuticos sem receita médica e programas de credenciamento de ciência de dados para fins lucrativos. Seus usuários começaram a reclamar e, portanto, o diretor do departamento de mensagens solicitou que você usasse a ciência de dados para descobrir uma maneira de filtrar essas mensagens de spam.

## Um filtro de spam realmente bobo

Imagine um "universo" que consiste em receber uma mensagem escolhida aleatoriamente de todas as mensagens possíveis. Seja $S$ o evento "a mensagem é spam" e $V$ é o evento "a mensagem contém a palavra "viagra". Em seguida, o Teorema de Bayes nos diz que a probabilidade de a mensagem ser spam condicionada a conter a palavra "viagra" é:

$$P(S~|~V) = \frac{P(V~|~S)~P(S)}{P(V~|~S)~P(S) ~+~ P(V~|~\neg S)~P(\neg S)}$$

O numerador é a probabilidade de uma mensagem ser spam **e** conter viagra, enquanto o denominador é apenas a probabilidade de uma mensagem conter viagra. Portanto, você pode pensar nesse cálculo como simplesmente representando a proporção de mensagens do viagra que são spam.

Se temos uma grande coleção de mensagens que sabemos serem spam, e uma grande coleção de mensagens que sabemos não ser spam, podemos estimar facilmente $P(V~|~S)$ e $P(V~|~\neg S)$. Se, além disso, assumirmos que qualquer mensagem tem a mesma probabilidade de ser spam ou não spam (de modo que $P(S) = P(\neg S) = 0.5$), então:

$$P(S~|~V) = \frac{P(V~|~S)~P(S)}{P(V~|~S) ~+~ P(V~|~\neg S)}$$

Por exemplo, se 50% das mensagens de spam tiverem a palavra viagra, mas apenas 1% das mensagens não spam tiverem, a probabilidade de um determinado email contendo viagra ser spam é:

$$P(S~|~V) = \frac{0.5}{0.5 + 0.01} = 98\%$$ 

## Um filtro de spam mais sofisticado

Imagine agora que temos um vocabulário de muitas palavras $w_1, \cdots, w_n$. Para mover isso para o reino da teoria da probabilidade, escreveremos $X_i$ para o evento "uma mensagem contém a palavra $w_i$". Imagine também que (através de algum processo ainda não especificado) nós chegamos a uma estimativa $P(X_i|S)$ para a probabilidade de que uma mensagem de spam contenha a iésima palavra e uma estimativa semelhante $P(X_i|\neg S)$ para a probabilidade de que uma mensagem não spam contenha a i-ésima palavra.

A chave para o algoritmo *Naive Bayes* é a suposição (forte) de que as presenças (ou ausências) de cada palavra são independentes umas das outras, dependendo de uma mensagem ser spam ou não. Intuitivamente, essa suposição significa que saber se uma certa mensagem de spam contém a palavra "viagra" não fornece informações sobre se essa mesma mensagem contém a palavra "rolex". Em termos matemáticos, isso significa que:

$$P(X_1 = x_1, \cdots, X_n = x_n~|~S) = P(X_1 = x_1~|~S) \times \cdots \times P(X_n = x_n~|~S)$$

Esta é uma suposição extrema. (Há uma razão pela qual a técnica tem um nome "*naive*".) Imagine que nosso vocabulário consiste apenas das palavras "viagra" e "rolex", e que metade de todas as mensagens de spam são para "viagra barato" e que a outra metade são para "rolex autêntico". Neste caso, a estimativa do Naive Bayes que uma mensagem de spam contém "viagra" e “rolex” é

$$P(X_1 = 1, X_2 = 1~|~S) = P(X_1 = 1~|~S)\times P(X_2 = 1~|~S) = 0.5 \times 0.5 = 0.25$$

uma vez que assumimos o conhecimento de que "viagra" e "rolex" nunca ocorrem juntos. Apesar do irrealismo desta suposição, este modelo freqüentemente executa bem e é usado em filtros de spam reais.

O mesmo raciocínio do Teorema de Bayes que usamos para o filtro de spam "*viagra-only*" nos diz que podemos calcular a probabilidade de uma mensagem ser spam usando a equação:

$$P(S~|~X = x) = \frac{P(X=x~|~S)}{P(X=x~|~S) + P(X=x~|~\neg S)}$$ 

A suposição Naive Bayes nos permite calcular cada uma das probabilidades à direita simplesmente multiplicando as estimativas de probabilidade individuais para cada palavra do vocabulário.

Na prática, você geralmente deseja evitar a multiplicação de muitas probabilidades, para evitar um problema chamado *underflow*, em que os computadores não lidam bem com números de pontos flutuantes muito próximos de zero. Lembrando da álgebra que $\log{ab} = \log{a} + \log{b}$ e que $\exp{\log{x}} = x$, usualmente computamos $p_1 \times \cdots \times p_n$ como a equivalente (mas "ponto flutuante mais amigável"):

$$\exp(~\log(p_1) + \cdots + \log(p_n)~)$$

O único desafio restante é encontrar estimativas para $P(X_i~|~S)$ e $P(X_i~|~\neg S)$, as probabilidades de que uma mensagem de spam (ou mensagem não spam) contenha a palavra $w_i$. Se tivermos um número razoável de mensagens de "treinamento" rotuladas como spam e não spam, uma primeira tentativa óbvia é estimar $P(X_i~|~S)$ simplesmente como a fração de mensagens de spam que contêm a palavra $w_i$.

Isso causa um grande problema, no entanto. Imagine que em nosso conjunto de treinamento a palavra "dados" do vocabulário só ocorre em mensagens não-spam. Em seguida, estimamos $P("data"~|~S) = 0$. O resultado é que nosso classificador Naive Bayes sempre atribuiria probabilidade de spam 0 a qualquer mensagem contendo a palavra "dados", até mesmo uma mensagem como "dados sobre viagra barato e relógios rolex autênticos". Para evitar esse problema, geralmente usamos algum tipo de suavização.

Em particular, escolheremos um *pseudo-contador - k -* e estimaremos a probabilidade de ver a i-ésima palavra em um spam como:

$$P(X_i~|~S) = \frac{k + n_{spams}(w_i)}{2 k + n_{spams}},$$

em que $n_{spams}$ é a quantidade de mensagens spam e $n_{spams}(w_i)$ é a quantidade de mensagens spam que contém a palavra $w_i$.

De forma análoga, calculamos $P(X_i~|~\neg S)$. Isto é, ao calcular as probabilidades de spam para a i-ésima palavra, assumimos que também vimos k spams adicionais contendo a palavra $w_i$ e $k$ spams adicionais que não contêm a palavra $w_i$.

Por exemplo, se "dados" ocorre em 0/98 documentos de spam, e se $k$ é $1$, estimamos $P("data"~|~S)$ como $1/100 = 0.01$, o que permite que nosso classificador ainda atribua alguma probabilidade de spam diferente de zero a mensagens que contenham a palavra "dados".

## Implementação

Agora temos todas as peças que precisamos para construir nosso classificador. Primeiro, vamos criar uma função simples para transformar as mensagens em palavras distintas. Primeiro, converteremos cada mensagem em minúscula. Para isso, use `re.findall()` para extrair "palavras" consistindo de letras, números e apóstrofos; e finalmente use `set()` para obter apenas as palavras distintas:

In [1]:
def tokenize(message):
    message = message.lower()                       # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message)   # extract the words
    return set(all_words)                           # remove duplicates

Nossa segunda função contará as palavras de um conjunto de treinamento de mensagens rotuladas. Teremos que retornar um dicionário cujas chaves sejam palavras e cujos valores sejam listas de dois elementos `[spam_count, non_spam_count]` correspondendo a quantas vezes vimos essa palavra em mensagens de spam e não-spam:

In [4]:
def count_words(training_set):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts

Nosso próximo passo é transformar essas contagens em probabilidades estimadas usando a suavização descrita anteriormente. Nossa função retornará uma lista de três valores: a palavra, a probabilidade de ver essa palavra em uma mensagem de spam e a probabilidade de ver essa palavra em uma mensagem não-spam:

In [5]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets
    w, p(w | spam) and p(w | ~spam)"""
    return [(w,
             (spam + k) / (total_spams + 2 * k),
             (non_spam + k) / (total_non_spams + 2 * k))
            for w, (spam, non_spam) in counts.items()]

A última parte é usar essas probabilidades de palavras (e nossas suposições Naive Bayes) para atribuir probabilidades às mensagens:

In [6]:
def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0

    for word, prob_if_spam, prob_if_not_spam in word_probs:

        # for each word in the message,
        # add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)

        # for each word that's not in the message
        # add the log probability of _not_ seeing it
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

Podemos colocar tudo isso junto em nosso classificador Naive Bayes:

In [7]:
class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def train(self, training_set):

        # count spam and non-spam messages
        num_spams = len([is_spam
                         for message, is_spam in training_set
                         if is_spam])
        num_non_spams = len(training_set) - num_spams

        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts,
                                             num_spams,
                                             num_non_spams,
                                             self.k)

    def classify(self, message):
        return spam_probability(self.word_probs, message)

## Testando nosso modelo

Um bom conjunto de dados (mas antigo) é o [corpus público do SpamAssassin](http://spamassassin.apache.org/old/publiccorpus/). Analisaremos os arquivos prefixados com *20021010*. 

Extraia os dados para a pasta `./data/spam/`. Depois de fazer isso, você deve ter três pastas: `spam`, `easy_ham` e `hard_ham`. Cada pasta contém muitos e-mails, cada um contido em um único arquivo. Para manter as coisas simples, vamos ver as linhas de assunto de cada e-mail.

Como identificamos a linha de assunto? Olhando através dos arquivos, todos parecem começar com "Assunto:". Então, vamos procurar por isso:

In [8]:
def get_subject_data(path):

    data = []

    # regex for stripping out the leading "Subject:" and any spaces after it
    subject_regex = re.compile(r"^Subject:\s+")

    # glob.glob returns every filename that matches the wildcarded path
    for fn in glob.glob(path):
        # diretorios com "ham" contém e-mails que não são spam
        is_spam = "ham" not in fn

        with open(fn,'r',encoding='ISO-8859-1') as file:
            for line in file:
                if line.startswith("Subject:"):
                    subject = subject_regex.sub("", line).strip()
                    data.append((subject, is_spam))

    return data

In [9]:
path = r".\data\spam\*\*"
data = get_subject_data(path)
print(len(data), "mensagens lidas.")

Agora a gente pode dividir os dados em conjunto de treino e conjunto de teste, e depois estaremos prontos para executar o classificador:

In [13]:
random.seed(0) # just so you get the same answers as me
train_data, test_data = split_data(data, 0.75)
classifier = NaiveBayesClassifier()
classifier.train(train_data)

Agora vamos ver como nosso modelo se saiu:

In [14]:
# triplets (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier.classify(subject))
              for subject, is_spam in test_data]

# assume that spam_probability > 0.5 corresponds to spam prediction
# and count the combinations of (actual is_spam, predicted is_spam)
counts = Counter((is_spam, spam_probability > 0.5)
                 for _, is_spam, spam_probability in classified)

print(counts)

Counter({(False, False): 704, (True, True): 101, (True, False): 38, (False, True): 33})


Isso dá 101 verdadeiros positivos (spam classificado como "spam"), 33 falsos positivos (*ham* classificado como "spam"), 704 negativos verdadeiros (*ham* classificado como "ham") e 38 falsos negativos (spam classificado como "*ham*"). Podemos armazenar esses valores: 

In [18]:
tp = counts[(True, True)]
fp = counts[(False, True)]
tn = counts[(False, False)]
fn = counts[(True, False)]

A nossa precisão é: $101 / (101 + 33) = 75\%$:

In [19]:
print("precisão: ", precision(tp, fp, fn, tn))

precisão:  0.753731343283582


O nosso *recall* é $101 / (101 + 38) = 73\%$:

In [20]:
print("revocação: ", recall(tp, fp, fn, tn))

revocação:  0.7266187050359713


Por fim, o nosso *f1-score* é $(2 \times 0.75 \times 0.73)/(0.75 + 0.73) = $

In [21]:
print("F1 score:", f1_score(tp, fp, fn, tn))

F1 score: 0.73992673992674


Esses não são números ruins para um modelo tão simples.

Também é interessante ver os mais erroneamente classificados:

In [24]:
classified.sort(key=lambda row: row[2])
spammiest_hams = list(filter(lambda row: not row[1], classified))[-5:]
hammiest_spams = list(filter(lambda row: row[1], classified))[:5]

In [25]:
print("spammiest_hams", spammiest_hams)

spammiest_hams [('Attn programmers: support offered [FLOSS-Sarai Initiative]', False, 0.9756129605140706), ('2000+ year old Greek computer reinterpreted', False, 0.9835355008104817), ('What to look for in your next smart phone (Tech Update)', False, 0.9898719206903398), ('[ILUG-Social] Re: Important - reenactor insurance needed', False, 0.9995349057803387), ('[ILUG-Social] Re: Important - reenactor insurance needed', False, 0.9995349057803387)]


In [26]:
print("hammiest_spams", hammiest_spams)

hammiest_spams [('Re: girls', True, 0.0009525186158413664), ('Introducing Chase Platinum for Students with a 0% Introductory APR', True, 0.0012566691211122248), ('.Message report from your contact page....//ytu855 rkq', True, 0.0015109358288642742), ('Testing a system, please delete', True, 0.00269205388369314), ('Never pay for the goodz again (8SimUgQ)', True, 0.005911623221945909)]


Os dois spammiest hams têm as palavras "necessário" (77 vezes mais chances de aparecer em spam), "seguro" (30 vezes mais chances de aparecer em spam) e "importante" (10 vezes mais chances de aparecer em spam) .

O spam mais antigo é muito curto ("Re: girls") para fazer muito julgamento, e o segundo mais difícil é uma solicitação de cartão de crédito cuja maioria das palavras não estava no conjunto de treinamento.

Podemos, da mesma forma, ver as palavras mais spam:

In [28]:
def p_spam_given_word(word_prob):
    word, prob_if_spam, prob_if_not_spam = word_prob
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [29]:
words = sorted(classifier.word_probs, key=p_spam_given_word)

spammiest_words = words[-5:]
hammiest_words = words[:5]

In [30]:
print("spammiest_words", spammiest_words)

spammiest_words [('year', 0.028767123287671233, 0.00022893772893772894), ('rates', 0.031506849315068496, 0.00022893772893772894), ('sale', 0.031506849315068496, 0.00022893772893772894), ('systemworks', 0.036986301369863014, 0.00022893772893772894), ('money', 0.03972602739726028, 0.00022893772893772894)]


In [31]:
print("hammiest_words", hammiest_words)

hammiest_words [('spambayes', 0.0013698630136986301, 0.04601648351648352), ('users', 0.0013698630136986301, 0.036401098901098904), ('razor', 0.0013698630136986301, 0.030906593406593408), ('zzzzteana', 0.0013698630136986301, 0.029075091575091576), ('sadev', 0.0013698630136986301, 0.026785714285714284)]


As palavras mais relacionadas com spam são "money", "systemworks", "rates", "sale" e "year", todas relacionadas à tentativa de levar as pessoas a comprar coisas. E as palavras mais relacionadas a mensagens não spam são "spambayes", "users", "razor", "zzzzteana" e "sadev", a maioria das quais parece relacionada à prevenção de spam, curiosamente.

### Como poderíamos obter um melhor desempenho?

Uma maneira óbvia seria obter mais dados para treinar. Há várias maneiras de melhorar o modelo também. Aqui estão algumas possibilidades que você pode tentar:

* Observe o conteúdo da mensagem, não apenas a linha de assunto. Você precisa ter cuidado ao lidar com os cabeçalhos das mensagens.

* Nosso classificador leva em conta todas as palavras que aparecem no conjunto de treinamento, mesmo as que aparecem apenas uma vez. Modifique o classificador para aceitar um limiar `min_count` opcional e ignorar os tokens que não aparecem pelo menos essa quantidade de vezes.

* O tokenizador não tem noção de palavras semelhantes (por exemplo, "cheap" e "cheapest"). Modifique o classificador para obter uma função opcional `stemmer` que converta palavras em *classes de equivalência*  de palavras. Por exemplo, uma função `stemmer` realmente simples pode ser:

In [32]:
def drop_final_s(word):
    return re.sub("s$", "", word)

In [34]:
print(drop_final_s("car"))
print(drop_final_s("cars"))

car
car


Criar uma boa função `stemmer` é difícil. As pessoas frequentemente usam o [Porter Stemmer](https://tartarus.org/martin/PorterStemmer/).

* Embora nossas *features* sejam todos da forma "mensagem contém a palavra $w_i$", não há motivo para que isso seja a regra. Em nossa implementação, podemos adicionar recursos extras, como "a mensagem contém um número", criando tokens falsos como `contains:number` e modificando o tokenizer para emiti-los quando apropriado.

## Para explorar

* Os artigos de Paul Graham, ["A Plan for Spam"](http://www.paulgraham.com/spam.html) e ["Better Bayesian Filtering"](http://www.paulgraham.com/better.html) (são interessantes e) dão mais informações sobre as ideias por trás da criação de filtros de spam.
* O [`scikit-learn`](http://scikit-learn.org/stable/modules/naive_bayes.html) contém um modelo `BernoulliNB` que implementa o mesmo algoritmo Naive Bayes que implementamos aqui, bem como outras variações do modelo.